In [69]:
import pandas as pd

In [70]:
aufwand_df = pd.read_csv('../../bexio/data/processed/aufwand.csv')
ertrag_df = pd.read_csv('../../bexio/data/processed/ertrag.csv')

In [71]:
def preprocess_dataframe(df):
    df['Datum'] = pd.to_datetime(df['Datum'])
    df['Year'] = df['Datum'].dt.year
    df['Month'] = df['Datum'].dt.month
    df['Quarter'] = df['Datum'].dt.quarter

    df_grouped = df.groupby(['Year', 'Month', 'Haben'])['Betrag'].sum().unstack('Haben')
    df_grouped = df_grouped.fillna(0)

    df_grouped_quarter = df.groupby(['Year', 'Quarter', 'Haben'])['Betrag'].sum().unstack('Haben')
    df_grouped_quarter = df_grouped_quarter.fillna(0)

    return df_grouped, df_grouped_quarter

aufwand_df_grouped, aufwand_df_grouped_quarter = preprocess_dataframe(aufwand_df)
ertrag_df_grouped, ertrag_df_grouped_quarter = preprocess_dataframe(ertrag_df)


In [72]:
ertrag_df_grouped

Haben       3200 - Handelserlös  3400 - Energieberatung  3402 - Planung HLK  \
Year Month                                                                    
2020 1                     0.00                    0.00              550.00   
     2                     0.00                    0.00             5322.24   
     3                     0.00                  300.00             3650.00   
     5                     0.00                  800.00             4300.00   
     6                     0.00                 1850.00              980.00   
     7                     0.00                 1600.00              550.00   
     8                     0.00                 1900.00              437.50   
     9                     0.00                 1100.00             4987.52   
     10                    0.00                    0.00              980.00   
     11                    0.00                 2590.02            10543.26   
     12                    0.00                    0.00             3990.02   
2021 1                     0.00                  800.00             8529.99   
     2                     0.00                  350.00            13097.40   
     3                     0.00                 1150.00            19215.07   
     4                     0.00                 4683.42             7657.52   
     5                     0.00                 5931.94            13027.57   
     6                     0.00                 1150.00            10510.07   
     7                     0.00                 6336.12                0.00   
     8                     0.00                 1448.46            15027.71   
     9                     0.00                13671.26             7670.01   
     10                    0.00                 1000.00                0.00   
     11                    0.00                 5334.68             1699.58   
     12                    0.00                 1850.00             5802.74   
2022 1                  9993.47                11472.70             3512.50   
     2                  1703.34                 8192.20             8747.50   
     3                     0.00                 4631.75                0.00   
     4                  6218.97                 8776.51            16786.60   
     5                  5081.87                 3753.28             1210.00   
     6                  1395.00                 4285.51             1515.00   
     7                     0.00                12661.93             8049.76   
     8                     0.00                 8304.55             1050.00   
     9                  3210.00                 8183.06            17897.27   
     10                    0.00                 4249.72            13167.00   
     11                    0.00                 1050.00             6159.59   
     12                    0.00                 6612.62             2480.00   
2023 1                     0.00                 5471.17             6035.00   
     2                     0.00                18678.73             5040.00   
     3                     0.00                13572.51                0.00   
     4                     0.00                35947.90                0.00   
     5                     0.00                10553.97             7946.86   
     6                     0.00                11513.47             3900.00   
     7                     0.00                  854.22                0.00   
     8                     0.00                13608.04                0.00   
     9                     0.00                 4131.85                0.00   
     11                    0.00                 1830.00                0.00   
     12                    0.00                 1500.00                0.00   

Haben       3600 - Gebühren Lieferungen und Leistungen  \
Year Month                                               
2020 1                                            0.00   
     2                           

In [73]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Load data
aufwand_df = pd.read_csv('../../src/bexio/data/processed/Aufwand.csv')
ertrag_df = pd.read_csv('../../src/bexio/data/processed/Ertrag.csv')


# Preprocess data
ertrag_df['Datum'] = pd.to_datetime(ertrag_df['Datum'])
ertrag_df['Year'] = ertrag_df['Datum'].dt.year
ertrag_df['Month'] = ertrag_df['Datum'].dt.month
ertrag_df['Quarter'] = ertrag_df['Datum'].dt.quarter

aufwand_df['Datum'] = pd.to_datetime(aufwand_df['Datum'])
aufwand_df['Year'] = aufwand_df['Datum'].dt.year
aufwand_df['Month'] = aufwand_df['Datum'].dt.month
aufwand_df['Quarter'] = aufwand_df['Datum'].dt.quarter

month_mapping = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May',
                 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October',
                 11: 'November', 12: 'December'}
quarter_mapping = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}

ertrag_df['Month'] = ertrag_df['Month'].map(month_mapping)
ertrag_df['Quarter'] = ertrag_df['Quarter'].map(quarter_mapping)
aufwand_df['Month'] = aufwand_df['Month'].map(month_mapping)
aufwand_df['Quarter'] = aufwand_df['Quarter'].map(quarter_mapping)


app = dash.Dash(__name__)

# Set default values
default_year = datetime.now().year
default_timeframe = 'Monthly'
default_stacked = True

app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in aufwand_df['Year'].unique()],
            value=default_year,
        )
    ]),
    html.Div([
        dcc.RadioItems(
            id="timeframe",
            options=[
                {"label": "Monthly", "value": 'Monthly'},
                {"label": "Quarterly", "value": 'Quarterly'}
            ],
            value=default_timeframe,
        ),
    ], style={'margin': '10px 0'}),
    html.Div([
        dcc.RadioItems(
            id="bar-mode",
            options=[
                {"label": "Grouped", "value": False},
                {"label": "Stacked", "value": True}
            ],
            value=default_stacked,
        ),
    ], style={'margin': '10px 0'}),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"),
    [Input("timeframe", "value"), Input("year-dropdown", "value"), Input("bar-mode", "value")])
def update_bar_chart(timeframe, year, stacked):
    if timeframe == 'Monthly':
        aufwand = aufwand_df[aufwand_df['Year'] == year].groupby(['Month', 'Haben'])['Betrag'].sum().unstack(fill_value=0)
        ertrag = ertrag_df[ertrag_df['Year'] == year].groupby(['Month', 'Haben'])['Betrag'].sum().unstack(fill_value=0)
    else:
        aufwand = aufwand_df[aufwand_df['Year'] == year].groupby(['Quarter', 'Haben'])['Betrag'].sum().unstack(fill_value=0)
        ertrag = ertrag_df[ertrag_df['Year'] == year].groupby(['Quarter', 'Haben'])['Betrag'].sum().unstack(fill_value=0)

    fig = go.Figure()
    if stacked:
        for column in aufwand.columns:
            fig.add_trace(go.Bar(x=aufwand.index,
                                 y=aufwand[column],
                                 name=f'Aufwand {column}',
                                 marker_color='rgb(55, 83, 109)'
                                 ))
        for column in ertrag.columns:
            fig.add_trace(go.Bar(x=ertrag.index,
                                 y=ertrag[column],
                                 name=f'Ertrag {column}',
                                 marker_color='rgb(26, 118, 255)',
                                 base=0,  # Set the base value for stacking
                                 offsetgroup='ertrag'  # Assign a unique group for stacking
                                 ))
        fig.update_layout(barmode='stack',  # Stack the "Ertrag" bars
                          bargroupgap=0.1  # Set the gap between the grouped "Aufwand" bars
                          )
    else:
        for column in aufwand.columns:
            fig.add_trace(go.Bar(x=aufwand.index,
                                 y=aufwand[column],
                                 name=f'Aufwand {column}',
                                 marker_color='rgb(55, 83, 109)'
                                 ))
        for column in ertrag.columns:
            fig.add_trace(go.Bar(x=ertrag.index,
                                 y=ertrag[column],
                                 name=f'Ertrag {column}',
                                 marker_color='rgb(26, 118, 255)'
                                 ))
        fig.update_layout(barmode='group')

    fig.update_layout(
        title=f'Aufwand vs Ertrag - {timeframe}',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Betrag',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        bargap=0.15,  # gap between bars of adjacent location coordinates.
        bargroupgap=0.1  # gap between bars of the same location coordinate.
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)


# Plot Aufwand

In [74]:
import datetime
import pandas as pd
import plotly.express as px

journal_df = pd.read_csv('../../bexio/data/raw/journal.csv')

journal_df = journal_df[~journal_df['Referenz'].str.contains('Rechnung')]
journal_df['Datum'] = pd.to_datetime(journal_df['Datum'], format='%d.%m.%Y')
aufwand_range = range(4000, 6941)
journal_df = journal_df[journal_df['Soll'].str.split(" - ").str[0].astype(int).isin(aufwand_range)]

current_date = datetime.date.today()
twelve_months_ago = current_date - datetime.timedelta(days=12 * 30)
twelve_months_ago = datetime.datetime.combine(twelve_months_ago, datetime.datetime.min.time())

filtered_df = journal_df[journal_df['Datum'] >= twelve_months_ago].copy()

# Combine year and month into a single column
filtered_df.loc[:, 'YearMonth'] = filtered_df['Datum'].dt.to_period('M').astype(str)

# Group by YearMonth and Soll category, and sum the amounts
grouped_df = filtered_df.groupby(['YearMonth', 'Soll'])['Betrag in Basiswährung'].sum().reset_index()

# Calculate the overall average
overall_average = grouped_df.groupby('YearMonth')['Betrag in Basiswährung'].sum().mean()

fig = px.bar(grouped_df, x='YearMonth', y='Betrag in Basiswährung', color='Soll',
             hover_data=['Betrag in Basiswährung'], labels={'Betrag in Basiswährung': 'Betrag in Basiswährung'}, height=400)

# Add the average line
fig.add_hline(y=overall_average, line_dash='dash', line_color='red', annotation_text=f'Average: {overall_average:.2f}')

fig.update_layout(xaxis={'type': 'category', 'categoryorder': 'category ascending'},
                  xaxis_title='Monate',
                  yaxis_title='Aufwand in CHF',
                  title='Aufwand der letzten 12 Monate (gem. Journal)')
fig.show()


# Plot Postkonto Verlauf

In [75]:
import pandas as pd
import plotly.express as px

journal_df = pd.read_csv('../../src/bexio/data/raw/journal.csv')

filtered_df = journal_df[journal_df['Soll'].str.contains('PostFinance', case=False) | journal_df['Haben'].str.contains('PostFinance', case=False)]

filtered_df.loc[filtered_df['Haben'].str.contains('PostFinance', case=False), 'Betrag in Basiswährung'] *= -1

filtered_df = filtered_df.copy()

filtered_df['Datum'] = pd.to_datetime(filtered_df['Datum'], format='%d.%m.%Y')

sorted_df = filtered_df.sort_values('Datum')

fig = px.bar(sorted_df, x='Datum', y='Betrag in Basiswährung', color='Soll',
             barmode='relative', labels={'Betrag in Basiswährung': 'Betrag in Basiswährung'})

fig.show()


In [76]:
#todo: doesnt make sense yet. current bank account balance should be aroun 50k
import pandas as pd
import plotly.express as px

filtered_df = filtered_df.copy()

filtered_df['Datum'] = pd.to_datetime(filtered_df['Datum'], format='%d.%m.%Y')

sorted_df = filtered_df.sort_values('Datum')

sorted_df['Cumulative Sum'] = sorted_df.loc[sorted_df['Soll'].str.contains('PostFinance', case=False) | sorted_df['Haben'].str.contains('PostFinance', case=False), 'Betrag in Basiswährung'].cumsum()

fig = px.line(sorted_df, x='Datum', y='Cumulative Sum')

fig.update_layout(title='Kumulative Summe von PostFinance',
                  xaxis_title='Datum',
                  yaxis_title='Kumulative Summe')

fig.show()
